In [1]:
!pip install stellargraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 435 kB 36.8 MB/s 


In [2]:
!pip install auto-sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 25.1 MB/s 
     |████████████████████████████████| 38.1 MB 9.6 MB/s 
     |████████████████████████████████| 22.3 MB 1.7 MB/s 
     |████████████████████████████████| 1.1 MB 11.8 MB/s 
     |████████████████████████████████| 837 kB 54.1 MB/s 
     |████████████████████████████████| 4.3 MB 44.4 MB/s 
     |████████████████████████████████| 4.4 MB 45.0 MB/s 
     |████████████████████████████████| 260 kB 53.5 MB/s 
     |████████████████████████████████| 140 kB 51.1 MB/s 
     |████████████████████████████████| 596 kB 23.7 MB/s 
     |████████████████████████████████| 46 kB 1.8 MB/s 
  Created wheel for auto-sklearn: filename=auto_sklearn-0.14.7-py3-none-any.whl size=6602873 sha256=f06bbffab9964775882f37d5ffa0cd045502db859fd945af898d21a9caa7ca5b
  Stored in directory: /root/.cache/pip/wheels/ba/43/5c/2fbe6fd19e3af314cbc4aa808378068d8ddd6792064f

In [58]:
# Importing Libraries
# please do go through this python notebook: 
import warnings
warnings.filterwarnings("ignore")

import csv
import pandas as pd # Pandas to create small dataframes 
import numpy as np
import datetime # Convert to unix time
import time # Convert to unix time
# If numpy is not installed already : pip3 install numpy
import numpy as np # Do aritmetic operations on arrays
# Matplotlib: used to plot graphs
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns # Plots
from matplotlib import rcParams # Size of plots  
import math
import pickle
import os
# To install xgboost: pip3 install xgboost
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

import warnings
import networkx as nx
import pdb
import pickle
from tqdm.notebook import tqdm
import os
import random
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
data_path = "gdrive/My Drive/GitHub Link Prediction/"

In [8]:
df_train = pd.read_csv(data_path+"gh_pos_train_91.csv").drop(columns = ['Unnamed: 0'])

In [9]:
df_train.head()

,src,dest,link
0,6639,22064,1
1,2040,26233,1
2,23636,36468,1
3,9723,17816,1
4,5068,16240,1


In [10]:
import tensorflow

In [11]:
from tensorflow import keras

In [12]:
from stellargraph.core import StellarGraph
from stellargraph.core import StellarDiGraph
from stellargraph.mapper import AdjacencyPowerGenerator
from stellargraph.layer import WatchYourStep
from stellargraph.losses import graph_log_likelihood
from stellargraph.utils import plot_history

from tensorflow.keras import optimizers, Model, layers, regularizers
import tensorflow as tf

In [13]:
from stellargraph.data import EdgeSplitter, UniformRandomWalk, UnsupervisedSampler
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, link_classification

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

In [14]:
train_graph = nx.from_pandas_edgelist(df_train, source='src', target='dest', create_using=nx.DiGraph())

In [16]:
degrees = nx.degree(train_graph)

In [18]:
degrees = dict(degrees)

In [19]:
import collections
features = {}
for key, val in degrees.items():
  features[key] = [val]

In [20]:
pr = nx.pagerank(train_graph, alpha=0.85)
pr = dict(pr)
for key, val in pr.items():
  features[key].append(val)

In [21]:
katz = nx.katz_centrality(train_graph,alpha=0.005,beta=1)
katz = dict(katz)
for key, val in katz.items():
  features[key].append(val)

In [22]:
features[0]

[1, 9.514840793661274e-06, 0.004748382141355886]

In [23]:
nx.set_node_attributes(train_graph, features, "features")

In [24]:
G = StellarGraph.from_networkx(train_graph, node_features="features")

In [25]:
print(G.info())

StellarDiGraph: Directed multigraph
 Nodes: 37700, Edges: 255678

 Node types:
  default: [37700]
    Features: float32 vector, length 3
    Edge types: default-default->default

 Edge types:
    default-default->default: [255678]
        Weights: all 1 (default)
        Features: none


In [26]:
nodes = list(G.nodes())
num_walks = 1
len = 5

In [27]:
unsupervised_samples = UnsupervisedSampler(
    G, nodes=nodes, length= len, number_of_walks = num_walks
)

In [28]:
batch_size=128
epochs=10
num_samples= [10,5]

In [29]:
generator = GraphSAGELinkGenerator(G, batch_size, num_samples)
train_gen = generator.flow(unsupervised_samples)

In [30]:
layer_sizes = [50,50]
graphsage = GraphSAGE(
    layer_sizes = layer_sizes, generator = generator, bias = True, normalize='l2'
)

In [31]:
x_inp, x_out = graphsage.in_out_tensors()

In [32]:
prediction = link_classification(
    output_dim = 1, output_act = "sigmoid", edge_embedding_method = "ip"
)(x_out)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [33]:
model = keras.Model(inputs = x_inp, outputs = prediction)
model.compile(
    optimizer = keras.optimizers.Adam(1e-2),
    loss = keras.losses.binary_crossentropy,
    metrics = [keras.metrics.binary_accuracy]
)

In [34]:
history = model.fit(
    train_gen,
    epochs=epochs,
    verbose=1,
    use_multiprocessing=True,
    shuffle=True,
)
filep = open(data_path+"graphsage_model.h5", "wb")
pickle.dump(model, filep)
filep.close()

Epoch 1/10
2357/2357 [==============================] - 231s 96ms/step - loss: 0.7163 - binary_accuracy: 0.5388
Epoch 2/10
2357/2357 [==============================] - 204s 86ms/step - loss: 0.6991 - binary_accuracy: 0.5563
Epoch 3/10
2357/2357 [==============================] - 205s 87ms/step - loss: 0.6962 - binary_accuracy: 0.5582
Epoch 4/10
2357/2357 [==============================] - 206s 87ms/step - loss: 0.6922 - binary_accuracy: 0.5629
Epoch 5/10
2357/2357 [==============================] - 204s 86ms/step - loss: 0.6903 - binary_accuracy: 0.5660
Epoch 6/10
2357/2357 [==============================] - 204s 87ms/step - loss: 0.6905 - binary_accuracy: 0.5651
Epoch 7/10
2357/2357 [==============================] - 202s 86ms/step - loss: 0.6918 - binary_accuracy: 0.5629
Epoch 8/10
2357/2357 [==============================] - 203s 86ms/step - loss: 0.6913 - binary_accuracy: 0.5631
Epoch 9/10
2357/2357 [==============================] - 203s 86ms/step - loss: 0.6901 - binary_accuracy:

In [35]:
x_inp_src = x_inp[0::2]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

In [36]:
from stellargraph.mapper.sampled_node_generators import GraphSAGENodeGenerator
node_ids = list(sorted(G.nodes()))
node_gen = GraphSAGENodeGenerator(G, batch_size, num_samples).flow(node_ids)

In [37]:
node_emb = embedding_model.predict(node_gen, verbose=1)

295/295 [==============================] - 14s 48ms/step


In [38]:
type(node_emb)

numpy.ndarray

In [39]:
node_emb.shape

(37700, 50)

In [40]:
node_emb[0].reshape(1,50).shape

(1, 50)

In [41]:
emb_dict = {}
for i, node in enumerate(node_ids):
  emb_dict[node] = np.array(node_emb[i].reshape(1,50))

In [42]:
combined = pd.read_csv(data_path+"/gh_train_91.csv", index_col = 0)
combined.head()

,src,dest,link
132913,10080,28269,1
97601,7104,31695,1
127442,21330,20345,0
189136,13326,30293,0
61075,13218,37521,0


In [43]:
# get training embeddings
train_emb = []
for i, row in tqdm(combined.iterrows()):
  comb_emb = (emb_dict[row['src']]) * (emb_dict[row['dest']])
  train_emb.append(comb_emb[0])
train_emb = np.array(train_emb)

0it [00:00, ?it/s]

In [44]:
train_y = combined['link']
train_y.shape

(516678,)

In [45]:
combined_test = pd.read_csv(data_path+"/gh_test_91.csv", index_col = 0)
combined_test.head()

,src,dest,link
222440,28906,32893,0
48306,35757,23700,0
265591,26731,28777,1
132547,5482,11624,0
124965,24379,459,0


In [46]:
# get test embeddings
test_emb = []
for i, row in tqdm(combined_test.iterrows()):
  comb_emb = (emb_dict[row['src']]) * (emb_dict[row['dest']])
  test_emb.append(comb_emb[0])

print(type(test_emb))
test_emb = np.array(test_emb)
print(test_emb.shape)
test_y = combined_test['link'] 

0it [00:00, ?it/s]

<class 'list'>
(62325, 50)


In [47]:
train_emb_sample, _, train_y_sample, _ = train_test_split(train_emb, train_y, stratify = train_y, test_size = 0.5, random_state = 42)

In [48]:
train_emb_sample.shape

(258339, 50)

In [49]:
from lightgbm import LGBMClassifier
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
clf.fit(train_emb, train_y)
train_sc = f1_score(train_y,clf.predict(train_emb))
test_sc = f1_score(test_y,clf.predict(test_emb))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.8429119554652653 test Score 0.834766516169174


In [50]:
#reading
from pandas import read_hdf
df_final_train = read_hdf(data_path + 'gh_fea_sample/gh_storage_sample_stage4.h5', 'train_df',mode='r')
df_final_test = read_hdf(data_path + 'gh_fea_sample/gh_storage_sample_stage4.h5', 'test_df',mode='r')

In [51]:
df_final_train.columns

Index(['src', 'dest', 'link', 'jaccard_followers', 'jaccard_followees',
       'cosine_followers', 'cosine_followees', 'num_followers_s',
       'num_followers_d', 'num_followees_s', 'num_followees_d',
       'inter_followers', 'inter_followees', 'adar_index', 'follows_back',
       'same_comp', 'shortest_path', 'weight_in', 'weight_out', 'weight_f1',
       'weight_f2', 'weight_f3', 'weight_f4', 'page_rank_s', 'page_rank_d',
       'katz_s', 'katz_d', 'hubs_s', 'hubs_d', 'authorities_s',
       'authorities_d', 'svd_u_s_1', 'svd_u_s_2', 'svd_u_s_3', 'svd_u_s_4',
       'svd_u_s_5', 'svd_u_s_6', 'svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3',
       'svd_u_d_4', 'svd_u_d_5', 'svd_u_d_6', 'svd_v_s_1', 'svd_v_s_2',
       'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6', 'svd_v_d_1',
       'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5', 'svd_v_d_6',
       'svd_dot_u', 'svd_dot_v', 'pref_attach_er', 'pref_attach_ee'],
      dtype='object')

In [52]:
df_final_train.shape

(516678, 59)

In [53]:
y_train = df_final_train.link
y_test = df_final_test.link

In [54]:
df_final_train.drop(['src', 'dest','link'],axis=1,inplace=True)
df_final_test.drop(['src', 'dest','link'],axis=1,inplace=True)

In [57]:
# GraphSAGE only
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(train_emb, train_y)
train_sc = f1_score(train_y,clf.predict(train_emb))
test_sc = f1_score(test_y,clf.predict(test_emb))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.8456946502342778 test Score 0.8347090284984434


In [59]:
# GraphSAGE only
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(train_emb, train_y)
train_sc = f1_score(train_y,clf.predict(train_emb))
test_sc = f1_score(test_y,clf.predict(test_emb))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.8513577300396621 test Score 0.8313725490196078


In [60]:
# GraphSAGE only
from lightgbm import LGBMClassifier
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
clf.fit(train_emb, train_y)
train_sc = f1_score(train_y,clf.predict(train_emb))
test_sc = f1_score(test_y,clf.predict(test_emb))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.8429119554652653 test Score 0.834766516169174


In [61]:
# Heuristic only
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9650688498879325 test Score 0.8361208524413272


In [62]:
# Heuristic only
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9586408046760307 test Score 0.8543679731353603


In [63]:
# Heuristic only
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.965836146176809 test Score 0.8412357286769645


In [64]:
df_final_train = np.hstack((df_final_train, train_emb))
print(df_final_train.shape)
df_final_test = np.hstack((df_final_test, test_emb))
print(df_final_test.shape)

(516678, 106)
(62325, 106)


In [65]:
print(type(train_y), type(y_train))
for i in range(len(y_train.values)):
  if train_y.values[i] != y_train.values[i]: print(i)

<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


TypeError: ignored

In [66]:
# Heuristic + GraphSAGE
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9653831300297641 test Score 0.835890863714546


In [67]:
# Heuristic + GraphSAGE
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9572501141253145 test Score 0.8616271307858876


In [68]:
# Heuristic + GraphSAGE
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.96573004560225 test Score 0.8420964304294561
